In [1]:
import gzip
import pandas as pd
import numpy as np
import subprocess
import copy
import subprocess

In [2]:
def read_delim_auto(filepath):
    """
    Automatically read .tsv, .csv, .tsv.gz, or .csv.gz based on extension.
    """
    filepath = str(filepath)
    if filepath.endswith(".tsv") or filepath.endswith(".tsv.gz"):
        return pd.read_csv(filepath, sep="\t", compression="infer")
    elif filepath.endswith(".csv") or filepath.endswith(".csv.gz"):
        return pd.read_csv(filepath, sep=",", compression="infer")
    else:
        raise ValueError(f"Unsupported file format: {filepath}")

In [3]:
def harmonise_and_format(
    sumstat,
    bim_file,
    has_rsid=True,
    n_cases=None,
    n_controls=None,
    rs_id = 'SNP',effect_allele = 'effect_allele',other_allele ='other_allele', eaf = 'effect_allele_frequency',
    beta = 'beta',se = 'standard_error',bp = 'base_pair_location',chromosome='chromosome',pval ='p_value'
):
   
    df = read_delim_auto(sumstat)
    df = df.copy()
 
    # Step 1: Rename SNP column for merge
    if has_rsid:
        df = df.rename(columns={rs_id: 'ID'})
        merged = bim_file.merge(df, on='ID')
    else:
        df['chr_pos'] = df[chromosome].astype(str) + '_' + df[bp].astype(str)
        merged = bim_file.merge(df, on='chr_pos')

    # Step 2: Harmonisation 
    # Adopted from https://github.com/zhwm/sharepro_coloc_analysis
    
    merged[effect_allele]= merged[effect_allele].str.upper()
    merged[other_allele]= merged[other_allele].str.upper()
    def assign_maf_beta(row):
        ea, oa = row[effect_allele], row[other_allele]
        alt, ref = row['ALT'], row['REF']
        eaf_val, beta_val = row[eaf], row[beta]
        if (ea, oa) == (alt, ref):
            return eaf_val, beta_val
        elif (ea, oa) == (ref, alt):
            return 1 - eaf_val, -beta_val
        else:
            return pd.NA, pd.NA

    merged[['eaf_temp', 'beta_temp']] = merged.apply(assign_maf_beta, axis=1, result_type='expand')
    merged['EAF'] = merged['eaf_temp']
    merged['BETA'] = merged['beta_temp']
    merged.drop(columns=['eaf_temp', 'beta_temp'], inplace=True)

    # Step 3: Add effective sample size
    if n_cases and n_controls:
        n_eff = round(4 / ((1 / n_cases) + (1 / n_controls)))
        merged['N'] = n_eff

    # Step 4: Rename columns using col_map
    renaming = {'ID':'SNP','ALT':'A1','REF':'A2',se:'SE',pval:'P'}
    merged = merged.rename(columns=renaming)

    selected_cols = ['SNP', 'A1', 'A2', 'EAF', 'BETA', 'SE', 'P', 'N']

    return merged[selected_cols]


In [4]:
def get_bim_dict(bimdir):
    # get bim, addopted from https://github.com/zhwm/sharepro_coloc_analysis
    bim_dict = {}
    bim_dup = []
    with open(bimdir) as bim:
        for line in bim:
            ll = line.strip().split()
            if ll[1] in bim_dict:
                bim_dup.append(ll[1])
            else:
                bim_dict[ll[1]]=(ll[1], ll[4], ll[5])
    # remove dups
    for drs in bim_dup:
        del bim_dict[drs]
    return bim_dict

In [5]:
bim_dic = get_bim_dict('/Users/a1/projects/IL6_pertrubation_coloc/sharepro_analysis/region_chr7_22466819_23071617.bim')

In [6]:
# Read exposure data
il6 = pd.read_table('../summ_stats/35459240-GCST90029070-EFO_0004458-Build37.f.tsv.gz')

In [7]:
il6 = il6.drop(['odds_ratio','ci_lower','ci_upper','effect_allele_frequency'], axis =1) # empty 
il6 = il6[il6.chromosome ==7]
il6 = il6.dropna()
il6 = il6.rename ( columns = {'variant_id':'ID'})


In [8]:
# To fill the eaf for IL6
frq_data = pd.read_table('../sharepro_analysis/region_chr7_22466819_23071617.afreq')

In [9]:
il6['alt_bim'] = il6['ID'].map(lambda x: bim_dic.get(x, (None,None,None))[1])
il6['ref_bim'] = il6['ID'].map(lambda x: bim_dic.get(x, (None,None,None))[2])


In [10]:
il6= il6[il6.ref_bim.notna()]
il6 = il6.merge(frq_data, on ='ID')
for index, row in il6.iterrows():
    if ((row['effect_allele'],row['other_allele'])==(row['ALT'], row['REF'])): 
        il6.at[index,'MAF'] = row['ALT_FREQS']
        il6.at[index, 'BETA'] = row['beta']
    elif ((row['effect_allele'],row['other_allele'])==(row['REF'], row['ALT'])):
        il6.at[index,'MAF'] = 1- row['ALT_FREQS']
        il6.at[index,'BETA'] = -row['beta']
il6[['N']] = 575531
st_il6 = il6[['ID','alt_bim','ref_bim','MAF','BETA','standard_error','p_value','N']]
st_il6= st_il6.rename(columns={'ID':'SNP','standard_error':'SE','alt_bim':'A1','ref_bim':'A2','p_value':'P',
                              'MAF':'EAF'})

In [11]:
# read bim file for harmonization
bim_file = pd.read_table('../sharepro_analysis/region_chr7_22466819_23071617.bim',header=None)
bim_file.columns = ['chr','ID','un','position','ALT','REF']
bim_file['chr_pos'] = bim_file.chr.astype(str) + '_'+ bim_file.position.astype(str)
bim_file = bim_file.drop_duplicates('position')

In [12]:

def intersect_by_bim(df1, df2, bim_dict):
    """
    Intersect two harmonised summary stats dataframes by SNPs present in a BIM dictionary.

    Parameters:
        df1 (pd.DataFrame): First summary stats dataframe (IL6).
        df2 (pd.DataFrame): Second summary stats dataframe (e.g., outcome).
        bim_dict (dict): Dictionary with SNPs (keys) to restrict intersection.

    Returns:
        df1_sub (pd.DataFrame), df2_sub (pd.DataFrame): Subsetted dataframes.
    """
    df1 = copy.deepcopy(df1)

    common_snps = set(df1.SNP) & set(df2.SNP)
    common_snps_filtered = [snp for snp in bim_dict.keys() if snp in common_snps]

    df1_sub = df1[df1['SNP'].isin(common_snps_filtered)].reset_index(drop=True)
    df2_sub = df2[df2['SNP'].isin(common_snps_filtered)].reset_index(drop=True)
    
    df1_sub.sort_values(by="SNP", key=lambda column: column.map(lambda e: common_snps_filtered.index(e)), inplace=True)
    df2_sub.sort_values(by="SNP", key=lambda column: column.map(lambda e: common_snps_filtered.index(e)), inplace=True)


    return df1_sub, df2_sub


In [14]:
import os

def run_pwcoco_pipeline(
    exposure_df,
    bim_file,
    bim_dict,
    outcomes_info: list,
    output_dir: str
):
    """
    Run harmonisation and intersection for multiple outcome files, saving results to a common directory.

    Parameters:
        exposure_df (pd.DataFrame): The IL6 (exposure) dataframe.
        bim_file (pd.DataFrame): BIM file dataframe.
        bim_dict (dict): Dictionary of SNPs from BIM.
        outcomes_info (list of dict): Each dict should contain:
            - 'path': path to outcome summary stats
            - 'n_cases': int
            - 'n_controls': int
            - 'has_rsid': bool
            - 'colnames': dict with column names
            - 'label': short label for output file naming
        output_dir (str): Directory where outputs will be saved.
    """
    os.makedirs(output_dir, exist_ok=True)

    for outcome in outcomes_info:
        print(f"Processing: {outcome['label']}")

        formatted_outcome = harmonise_and_format(
            sumstat=outcome['path'],
            bim_file=bim_file,
            has_rsid=outcome.get('has_rsid', True),
            n_cases=outcome['n_cases'],
            n_controls=outcome['n_controls'],
            **outcome.get('colnames', {})
        )

        st_exp, st_out = intersect_by_bim(exposure_df, formatted_outcome, bim_dict)

        st_exp.to_csv(os.path.join(output_dir, f"IL6_{outcome['label']}.pwcoco"), sep="\t", index=False)
        st_out.to_csv(os.path.join(output_dir, f"{outcome['label']}.pwcoco"), sep="\t", index=False)
        st_exp[['SNP']].to_csv(os.path.join(output_dir, f"IL6_{outcome['label']}_forLD.txt"), sep="\t", index=False, header=False)

In [15]:
outcomes_info = [
    {
        'path': '../summ_stats/LAS.tsv.gz',
        'n_cases': 9219,
        'n_controls': 1503898,
        'has_rsid': False,
        'label': 'LAS',
    },
    {
        'path': '../summ_stats/CAD_chr7_GCST90132315.tsv',
        'n_cases': 210842,
        'n_controls': 1167328,
        'has_rsid': False,
        'label': 'CAD',
        'colnames': {
            'effect_allele': 'effect_allele',
            'other_allele': 'other_allele',
            'eaf': 'effect_allele_frequency',
            'beta': 'beta',
            'se': 'standard_error',
            'bp': 'base_pair_location',
            'chromosome': 'chromosome',
            'pval': 'p_value'
        }
    },
    
    {
        'path': '../summ_stats/IS_GCST90104535_buildGRCh37.tsv.gz',
        'n_cases': 86668,
        'n_controls': 1503898,
        'has_rsid': False,
        'label': 'IS'
    },
    
    {
        'path': '../summ_stats/PAD.csv',
        'n_cases': 31307,
        'n_controls': 211753,
        'has_rsid': True,
        'label': 'PAD',
        'colnames': {'rs_id':'SNP',
            'effect_allele': 'effect_allele.outcome',
            'other_allele': 'other_allele.outcome',
            'eaf': 'eaf.outcome',
            'beta': 'beta.outcome',
            'se': 'se.outcome',
            'bp': 'pos.outcome',
            'chromosome': 'chr.outcome',
            'pval': 'pval.outcome'
        }
    },
    {
        'path': '../summ_stats/plaque.tsv',
        'n_cases': 21540,
        'n_controls': 26894,
        'has_rsid': True,
        'label': 'PL',
        'colnames': {'rs_id':'rs_id'}
    },
    
    {
        'path': '../summ_stats/RA_processed.tsv',
        'n_cases': 35871,
        'n_controls': 240149,
        'has_rsid': True,
        'label': 'RA',
        'colnames': {'rs_id':'ID',
            'eaf': 'effect_allele_frequency',
            'se': 'standard_error',
            'bp': 'position',
            'chromosome': 'chr',
            'pval': 'p_value'
        }
    },
    {
        'path': '../summ_stats/Meta_PMR_chr7.tsv',
        'n_cases': 8156,
        'n_controls': 416495,
        'has_rsid': True,
        'label': 'PMR',
        'colnames': {'rs_id':'SNP',
            'effect_allele': 'EA',
            'other_allele': 'NEA',
            'eaf': 'EAF',
            'beta': 'Beta',
            'se': 'StdErr',
            'bp': 'position',
            'chromosome': 'chr',
            'pval': 'Pvalue'
        }
    }
    
]

In [16]:
output_dir = "pwcoco_outputs"
run_pwcoco_pipeline(st_il6, bim_file, bim_dic, outcomes_info, output_dir)

Processing: LAS
Processing: CAD
Processing: IS
Processing: PAD
Processing: PL
Processing: RA
Processing: PMR


## Preprare an LD matrix for every outcome using: 

#### plink --bfile region_chr7_22466819_23071617 --extract IL6_{outcome}_forLD.txt \
#### --matrix --out IL6_{outcome}_LD --r --write-snplist

it will generate a file 'IL6_{outcome}_LD.ld' which should be passed to sharepro

In [17]:
# List of outcome names
names = ['CAD', 'IS', 'LAS', 'PAD', 'PL', 'PMR', 'RA']

for name in names:
    # run sharepro coloc
    command = f"python ~/projects/SharePro_coloc/src/SharePro/sharepro_coloc.py --z pwcoco_outputs/IL6_{name}.pwcoco pwcoco_outputs/{name}.pwcoco --ld /Users/a1/projects/IL6_pertrubation_coloc/sharepro_analysis/IL6_{name}_LD.ld --save results_test/resIL6_{name}"

    subprocess.run(command, shell=True, check=True)
    print(f"Executed for {name}")


Executed for CAD
Executed for IS
Executed for LAS
Executed for PAD
Executed for PL
Executed for PMR
Executed for RA
